# GIFT-Riemann Phase 1 : Hybrid Lags + Log-Correction

## GPU-Accelerated Validation (CuPy/A100)

Ce notebook implémente les tests prioritaires identifiés par le conseil des IAs :

### Tests Implémentés

| Test | Question | GPU Acceleration |
|------|----------|------------------|
| **1. Hybrid Lags** | [1,2,3,4] + GIFT lag optimal ? | Parallel search |
| **2. Log-Correction** | aᵢ(n) = aᵢ^∞ + bᵢ/log(n) ? | Vectorized fitting |
| **3. GIFT Asymptotic** | Limites → constantes GIFT ? | Batch regression |
| **4. Stability Map** | Coefficients par fenêtre | Parallel windows |

### Objectifs

- Trouver la combinaison hybride optimale
- Vérifier si les coefficients convergent vers GIFT quand n→∞
- Produire une carte de stabilité des coefficients

In [ ]:
# Installation si nécessaire (Colab)
# !pip install cupy-cuda12x  # Pour CUDA 12.x
# !pip install cupy-cuda11x  # Pour CUDA 11.x

In [ ]:
import numpy as np
import json
import time
from typing import List, Tuple, Dict, Optional
from itertools import combinations

# GPU Setup
try:
    import cupy as cp
    from cupyx.scipy.linalg import lstsq as cp_lstsq
    GPU_AVAILABLE = True
    print(f"✅ CuPy disponible - GPU: {cp.cuda.runtime.getDeviceCount()} device(s)")
    print(f"   Device: {cp.cuda.runtime.getDeviceProperties(0)['name'].decode()}")
    mempool = cp.get_default_memory_pool()
except ImportError:
    GPU_AVAILABLE = False
    print("⚠️  CuPy non disponible - fallback NumPy")
    cp = np

# Adaptive backend
xp = cp if GPU_AVAILABLE else np
print(f"\nBackend: {'CuPy (GPU)' if GPU_AVAILABLE else 'NumPy (CPU)'}")

## 1. Chargement des Zéros

Uploadez vos fichiers de zéros (zeros1, zeros2, etc.) ou utilisez le file picker.

In [ ]:
def load_zeros_from_files(file_paths: List[str]) -> np.ndarray:
    """
    Charge les zéros depuis plusieurs fichiers.
    Supporte différents formats (une valeur par ligne).
    """
    all_zeros = []
    
    for path in file_paths:
        print(f"Chargement: {path}")
        try:
            with open(path, 'r') as f:
                for line in f:
                    line = line.strip()
                    # Skip comments and headers
                    if not line or line.startswith('#') or line.startswith('Values'):
                        continue
                    try:
                        val = float(line.split()[0])  # Premier nombre de la ligne
                        if val > 0:  # Zéros positifs uniquement
                            all_zeros.append(val)
                    except (ValueError, IndexError):
                        continue
            print(f"  → {len(all_zeros)} zéros cumulés")
        except FileNotFoundError:
            print(f"  ⚠️ Fichier non trouvé: {path}")
    
    # Trier et dédupliquer
    all_zeros = sorted(set(all_zeros))
    return np.array(all_zeros)

# Pour Colab: upload interactif
def load_zeros_colab():
    """Upload interactif pour Google Colab."""
    try:
        from google.colab import files
        print("Uploadez vos fichiers de zéros...")
        uploaded = files.upload()
        
        all_zeros = []
        for filename, content in uploaded.items():
            print(f"Traitement: {filename}")
            lines = content.decode('utf-8').split('\n')
            for line in lines:
                line = line.strip()
                if line and not line.startswith('#'):
                    try:
                        val = float(line.split()[0])
                        if val > 0:
                            all_zeros.append(val)
                    except:
                        continue
        
        all_zeros = sorted(set(all_zeros))
        print(f"\nTotal: {len(all_zeros)} zéros chargés")
        return np.array(all_zeros)
    except ImportError:
        print("Pas sur Colab - utilisez load_zeros_from_files()")
        return None

In [ ]:
# ============================================================
# CHARGEMENT DES DONNÉES
# ============================================================

# Option 1: Fichiers locaux
gamma_np = load_zeros_from_files(['zeros1'])  # Ajoutez vos fichiers ici

# Option 2: Colab upload (décommenter)
# gamma_np = load_zeros_colab()

# Transfert GPU
gamma = xp.asarray(gamma_np)
N_ZEROS = len(gamma)

print(f"\n{'='*60}")
print(f"DONNÉES CHARGÉES")
print(f"{'='*60}")
print(f"Nombre de zéros: {N_ZEROS:,}")
print(f"Plage: γ₁ = {float(gamma[0]):.4f} à γ_{N_ZEROS} = {float(gamma[-1]):.4f}")
print(f"Stockage: {gamma.nbytes / 1e6:.2f} MB sur {'GPU' if GPU_AVAILABLE else 'CPU'}")

## 2. Fonctions Utilitaires GPU

In [ ]:
def local_spacing_vec(T: xp.ndarray) -> xp.ndarray:
    """
    Espacement moyen local (vectorisé).
    Δ(T) ≈ 2π / log(T / 2π)
    """
    two_pi = 2 * xp.pi
    safe_T = xp.maximum(T, two_pi + 1e-10)
    return two_pi / xp.log(safe_T / two_pi)


def fit_recurrence_gpu(gamma: xp.ndarray, lags: List[int], 
                       start: int = None, end: int = None) -> Tuple[xp.ndarray, float, float]:
    """
    Fit récurrence linéaire sur GPU.
    γₙ = Σᵢ aᵢ γₙ₋ₗᵢ + c
    
    Returns: (coefficients, mean_rel_error_pct, mean_unfolded_error)
    """
    max_lag = max(lags)
    if start is None:
        start = max_lag
    if end is None:
        end = len(gamma)
    
    n_points = end - start
    n_params = len(lags) + 1  # +1 pour constante
    
    # Construction matricielle vectorisée
    indices = xp.arange(start, end)
    
    # X matrix: chaque colonne = gamma[n - lag] pour chaque lag
    X = xp.zeros((n_points, n_params), dtype=xp.float64)
    for i, lag in enumerate(lags):
        X[:, i] = gamma[indices - lag]
    X[:, -1] = 1.0  # Colonne constante
    
    # y vector
    y = gamma[indices]
    
    # Least squares (GPU ou CPU)
    if GPU_AVAILABLE:
        # CuPy lstsq
        coeffs, residuals, rank, s = xp.linalg.lstsq(X, y, rcond=None)
    else:
        coeffs, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
    
    # Prédictions et erreurs
    y_pred = X @ coeffs
    errors_abs = xp.abs(y_pred - y)
    errors_rel_pct = errors_abs / y * 100
    
    # Erreur unfolded
    spacings = local_spacing_vec(y)
    errors_unfolded = errors_abs / spacings
    
    mean_rel = float(xp.mean(errors_rel_pct))
    mean_unf = float(xp.mean(errors_unfolded))
    
    return coeffs, mean_rel, mean_unf


def clear_gpu_memory():
    """Libère la mémoire GPU."""
    if GPU_AVAILABLE:
        mempool.free_all_blocks()
        cp.cuda.Stream.null.synchronize()

---

# TEST 1 : Recherche Hybride Optimale

Tester toutes les combinaisons [1,2,3,4] + 1 lag GIFT supplémentaire.

In [ ]:
def test_hybrid_lags(gamma: xp.ndarray, base_lags: List[int] = [1, 2, 3, 4],
                     additional_lags: List[int] = None) -> Dict:
    """
    TEST 1: Recherche de la combinaison hybride optimale.
    
    Teste: base_lags + chaque lag additionnel
    """
    print("\n" + "="*60)
    print("TEST 1 : RECHERCHE HYBRIDE OPTIMALE")
    print("="*60)
    
    if additional_lags is None:
        # Lags GIFT et voisins intéressants
        additional_lags = [5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 
                          20, 21, 22, 27, 28, 30, 35, 40]
    
    results = []
    
    # Baseline: juste les lags de base
    print(f"\n📊 Baseline {base_lags}...")
    coeffs_base, err_pct_base, err_unf_base = fit_recurrence_gpu(gamma, base_lags)
    results.append({
        'lags': base_lags.copy(),
        'extra': None,
        'error_pct': err_pct_base,
        'error_unf': err_unf_base,
        'coeffs': [float(c) for c in coeffs_base]
    })
    print(f"   Erreur: {err_pct_base:.4f}% / {err_unf_base:.4f} spacings")
    
    # Test avec chaque lag additionnel
    print(f"\n🔍 Test hybrides {base_lags} + [x]...")
    
    for extra_lag in additional_lags:
        if extra_lag in base_lags:
            continue
        
        hybrid_lags = sorted(base_lags + [extra_lag])
        
        try:
            coeffs, err_pct, err_unf = fit_recurrence_gpu(gamma, hybrid_lags)
            
            improvement = (err_unf_base - err_unf) / err_unf_base * 100
            
            results.append({
                'lags': hybrid_lags,
                'extra': extra_lag,
                'error_pct': err_pct,
                'error_unf': err_unf,
                'improvement': improvement,
                'coeffs': [float(c) for c in coeffs]
            })
            
            status = "★" if improvement > 5 else "✓" if improvement > 0 else ""
            print(f"   +[{extra_lag:2d}] → {err_unf:.4f} spacings ({improvement:+.1f}%) {status}")
            
        except Exception as e:
            print(f"   +[{extra_lag:2d}] → Erreur: {e}")
    
    # Test avec 2 lags additionnels (GIFT pairs)
    print(f"\n🔍 Test hybrides {base_lags} + [x, y]...")
    gift_pairs = [
        [5, 8], [5, 13], [5, 27], [8, 13], [8, 14], [8, 21], [8, 27],
        [13, 27], [14, 21], [14, 27], [21, 27]
    ]
    
    for pair in gift_pairs:
        hybrid_lags = sorted(base_lags + pair)
        
        try:
            coeffs, err_pct, err_unf = fit_recurrence_gpu(gamma, hybrid_lags)
            improvement = (err_unf_base - err_unf) / err_unf_base * 100
            
            results.append({
                'lags': hybrid_lags,
                'extra': pair,
                'error_pct': err_pct,
                'error_unf': err_unf,
                'improvement': improvement,
                'coeffs': [float(c) for c in coeffs]
            })
            
            status = "★" if improvement > 10 else "✓" if improvement > 0 else ""
            print(f"   +{pair} → {err_unf:.4f} spacings ({improvement:+.1f}%) {status}")
            
        except Exception as e:
            print(f"   +{pair} → Erreur: {e}")
    
    # Tri par erreur
    results_sorted = sorted(results, key=lambda x: x['error_unf'])
    
    # Top 10
    print(f"\n🏆 TOP 10 COMBINAISONS:")
    print(f"{'Rank':<5} {'Lags':<25} {'Err(%)':>10} {'Err(spac)':>12} {'Improv':>10}")
    print("-" * 65)
    
    for i, r in enumerate(results_sorted[:10]):
        lags_str = str(r['lags'])
        improv = r.get('improvement', 0)
        print(f"{i+1:<5} {lags_str:<25} {r['error_pct']:>10.4f} {r['error_unf']:>12.4f} {improv:>+10.1f}%")
    
    # Meilleur résultat
    best = results_sorted[0]
    print(f"\n🎯 MEILLEURE COMBINAISON: {best['lags']}")
    print(f"   Erreur: {best['error_pct']:.4f}% / {best['error_unf']:.4f} spacings")
    
    clear_gpu_memory()
    
    return {
        'best': best,
        'baseline': results[0],
        'all_results': results_sorted
    }

# Exécution
hybrid_results = test_hybrid_lags(gamma)

---

# TEST 2 : Coefficients Log-Dépendants

Tester si aᵢ(n) = aᵢ^∞ + bᵢ/log(n) stabilise les coefficients.

In [ ]:
def test_log_dependent_coefficients(gamma: xp.ndarray, 
                                     lags: List[int] = [5, 8, 13, 27],
                                     n_windows: int = 20,
                                     window_size: int = None) -> Dict:
    """
    TEST 2: Analyse des coefficients par fenêtres glissantes.
    
    Vérifie si aᵢ(n) = aᵢ^∞ + bᵢ/log(n)
    """
    print("\n" + "="*60)
    print("TEST 2 : COEFFICIENTS LOG-DÉPENDANTS")
    print("="*60)
    
    N = len(gamma)
    max_lag = max(lags)
    
    if window_size is None:
        window_size = (N - max_lag) // n_windows
    
    print(f"\n📊 Configuration:")
    print(f"   Lags: {lags}")
    print(f"   Fenêtres: {n_windows} × {window_size} points")
    
    # Extraction des coefficients par fenêtre
    window_data = []
    
    print(f"\n🔄 Extraction des coefficients par fenêtre...")
    
    for i in range(n_windows):
        start = max_lag + i * window_size
        end = start + window_size
        
        if end > N:
            break
        
        # Indice central et gamma central
        n_center = (start + end) // 2
        gamma_center = float(gamma[n_center])
        log_n = np.log(n_center)
        
        # Fit sur cette fenêtre
        coeffs, err_pct, err_unf = fit_recurrence_gpu(gamma, lags, start=start, end=end)
        
        window_data.append({
            'window': i,
            'n_center': n_center,
            'gamma_center': gamma_center,
            'log_n': log_n,
            'inv_log_n': 1.0 / log_n,
            'coeffs': [float(c) for c in coeffs],
            'error_pct': err_pct,
            'error_unf': err_unf
        })
        
        if i % 5 == 0:
            print(f"   Fenêtre {i+1}/{n_windows}: n={n_center}, err={err_unf:.4f} spac")
    
    # Analyse: fit linéaire aᵢ vs 1/log(n)
    print(f"\n📈 Régression aᵢ(n) = aᵢ^∞ + bᵢ/log(n)...")
    
    inv_log_n = np.array([w['inv_log_n'] for w in window_data])
    n_params = len(lags) + 1
    
    regression_results = []
    
    # GIFT theoretical values
    gift_theory = {
        5: 0.5,           # N_gen/h_G₂ = 3/6
        8: 56/99,         # fund(E₇)/H*
        13: -14/99,       # -dim(G₂)/H*
        27: 1/27,         # 1/dim(J₃𝕆)
        'c': 99/5         # H*/Weyl
    }
    
    for j in range(n_params):
        coeff_values = np.array([w['coeffs'][j] for w in window_data])
        
        # Régression linéaire: coeff = a_inf + b * (1/log(n))
        X_reg = np.column_stack([np.ones_like(inv_log_n), inv_log_n])
        params, residuals, rank, s = np.linalg.lstsq(X_reg, coeff_values, rcond=None)
        
        a_inf = params[0]  # Limite asymptotique
        b = params[1]      # Correction log
        
        # R² score
        y_pred = X_reg @ params
        ss_res = np.sum((coeff_values - y_pred)**2)
        ss_tot = np.sum((coeff_values - np.mean(coeff_values))**2)
        r_squared = 1 - ss_res / ss_tot if ss_tot > 0 else 0
        
        # Coefficient de variation
        cv = np.std(coeff_values) / np.abs(np.mean(coeff_values)) * 100 if np.mean(coeff_values) != 0 else float('inf')
        
        # Comparaison GIFT
        if j < len(lags):
            lag = lags[j]
            name = f"a_{lag}"
            gift_val = gift_theory.get(lag, None)
        else:
            name = "c"
            gift_val = gift_theory.get('c', None)
        
        gift_diff = abs(a_inf - gift_val) / abs(gift_val) * 100 if gift_val else None
        
        regression_results.append({
            'name': name,
            'a_inf': a_inf,
            'b': b,
            'r_squared': r_squared,
            'cv_raw': cv,
            'gift_theory': gift_val,
            'gift_diff_pct': gift_diff,
            'values': coeff_values.tolist()
        })
    
    # Affichage
    print(f"\n{'Coeff':<8} {'a_inf':>10} {'b':>10} {'R²':>8} {'CV(%)':>8} {'GIFT':>10} {'Écart':>10}")
    print("-" * 75)
    
    for r in regression_results:
        gift_str = f"{r['gift_theory']:.4f}" if r['gift_theory'] else "N/A"
        diff_str = f"{r['gift_diff_pct']:.1f}%" if r['gift_diff_pct'] else "N/A"
        print(f"{r['name']:<8} {r['a_inf']:>10.4f} {r['b']:>10.2f} {r['r_squared']:>8.3f} {r['cv_raw']:>8.1f} {gift_str:>10} {diff_str:>10}")
    
    # Test de stabilisation
    print(f"\n🎯 ANALYSE:")
    
    high_r2 = sum(1 for r in regression_results if r['r_squared'] > 0.5)
    close_to_gift = sum(1 for r in regression_results if r['gift_diff_pct'] and r['gift_diff_pct'] < 20)
    
    print(f"   Coefficients avec R² > 0.5 (log-dépendance): {high_r2}/{len(regression_results)}")
    print(f"   Coefficients proches GIFT (<20%): {close_to_gift}/{len(regression_results)}")
    
    if high_r2 >= 3:
        print(f"\n   ✅ STRUCTURE LOG-DÉPENDANTE CONFIRMÉE")
        verdict = "PASS"
    elif high_r2 >= 1:
        print(f"\n   ⚠️  STRUCTURE PARTIELLE")
        verdict = "MARGINAL"
    else:
        print(f"\n   ❌ PAS DE STRUCTURE LOG-DÉPENDANTE CLAIRE")
        verdict = "FAIL"
    
    clear_gpu_memory()
    
    return {
        'verdict': verdict,
        'regression': regression_results,
        'window_data': window_data,
        'high_r2_count': high_r2,
        'close_to_gift_count': close_to_gift
    }

# Exécution
log_results = test_log_dependent_coefficients(gamma, lags=[5, 8, 13, 27], n_windows=20)

---

# TEST 3 : Carte de Stabilité des Coefficients

Visualisation de l'évolution des coefficients sur toute la plage.

In [ ]:
def stability_map(gamma: xp.ndarray, lags: List[int] = [5, 8, 13, 27],
                  n_windows: int = 50) -> Dict:
    """
    TEST 3: Carte de stabilité des coefficients.
    """
    print("\n" + "="*60)
    print("TEST 3 : CARTE DE STABILITÉ")
    print("="*60)
    
    N = len(gamma)
    max_lag = max(lags)
    window_size = (N - max_lag) // n_windows
    
    print(f"\n📊 Calcul de {n_windows} fenêtres...")
    
    stability_data = []
    
    for i in range(n_windows):
        start = max_lag + i * window_size
        end = start + window_size
        
        if end > N:
            break
        
        coeffs, err_pct, err_unf = fit_recurrence_gpu(gamma, lags, start=start, end=end)
        
        stability_data.append({
            'n_center': (start + end) // 2,
            'coeffs': {f'a_{lag}': float(coeffs[j]) for j, lag in enumerate(lags)},
            'c': float(coeffs[-1]),
            'error_unf': err_unf
        })
    
    # Statistiques de stabilité
    print(f"\n📈 Statistiques de stabilité:")
    print(f"{'Coeff':<8} {'Mean':>10} {'Std':>10} {'CV(%)':>10} {'Min':>10} {'Max':>10}")
    print("-" * 60)
    
    stability_stats = {}
    
    for j, lag in enumerate(lags):
        key = f'a_{lag}'
        values = [d['coeffs'][key] for d in stability_data]
        mean = np.mean(values)
        std = np.std(values)
        cv = std / abs(mean) * 100 if mean != 0 else float('inf')
        
        stability_stats[key] = {
            'mean': mean, 'std': std, 'cv': cv,
            'min': min(values), 'max': max(values)
        }
        
        print(f"{key:<8} {mean:>10.4f} {std:>10.4f} {cv:>10.1f} {min(values):>10.4f} {max(values):>10.4f}")
    
    # Constante
    c_values = [d['c'] for d in stability_data]
    mean_c = np.mean(c_values)
    std_c = np.std(c_values)
    cv_c = std_c / abs(mean_c) * 100 if mean_c != 0 else float('inf')
    stability_stats['c'] = {'mean': mean_c, 'std': std_c, 'cv': cv_c}
    print(f"{'c':<8} {mean_c:>10.4f} {std_c:>10.4f} {cv_c:>10.1f} {min(c_values):>10.4f} {max(c_values):>10.4f}")
    
    # Erreur moyenne
    mean_err = np.mean([d['error_unf'] for d in stability_data])
    print(f"\n   Erreur unfolded moyenne: {mean_err:.4f} spacings")
    
    clear_gpu_memory()
    
    return {
        'data': stability_data,
        'stats': stability_stats
    }

# Exécution
stability_results = stability_map(gamma, lags=[5, 8, 13, 27], n_windows=50)

---

# TEST 4 : Comparaison GIFT Pure vs Hybride vs [1,2,3,4]

In [ ]:
def compare_all_approaches(gamma: xp.ndarray) -> Dict:
    """
    TEST 4: Comparaison complète de toutes les approches.
    """
    print("\n" + "="*60)
    print("TEST 4 : COMPARAISON COMPLÈTE")
    print("="*60)
    
    approaches = [
        ([1, 2, 3, 4], "Consécutifs"),
        ([5, 8, 13, 27], "GIFT Fibonacci"),
        ([3, 5, 8, 13], "Fibonacci pur"),
        ([8, 14, 21, 27], "GIFT constants"),
        ([1, 2, 3, 4, 8], "Hybrid +8"),
        ([1, 2, 3, 4, 14], "Hybrid +14"),
        ([1, 2, 3, 4, 21], "Hybrid +21"),
        ([1, 2, 3, 4, 27], "Hybrid +27"),
        ([1, 2, 3, 4, 8, 14], "Hybrid +8+14"),
        ([1, 2, 3, 4, 8, 27], "Hybrid +8+27"),
        ([1, 2, 3, 4, 5, 8, 13, 27], "Full Hybrid"),
    ]
    
    results = []
    
    print(f"\n{'Approche':<20} {'Lags':<25} {'Err(%)':>10} {'Err(spac)':>12}")
    print("-" * 70)
    
    for lags, name in approaches:
        try:
            coeffs, err_pct, err_unf = fit_recurrence_gpu(gamma, lags)
            results.append({
                'name': name,
                'lags': lags,
                'error_pct': err_pct,
                'error_unf': err_unf,
                'coeffs': [float(c) for c in coeffs]
            })
            print(f"{name:<20} {str(lags):<25} {err_pct:>10.4f} {err_unf:>12.4f}")
        except Exception as e:
            print(f"{name:<20} {str(lags):<25} {'ERROR':>10}")
    
    # Classement
    results_sorted = sorted(results, key=lambda x: x['error_unf'])
    
    print(f"\n🏆 CLASSEMENT:")
    for i, r in enumerate(results_sorted[:5]):
        print(f"   {i+1}. {r['name']}: {r['error_unf']:.4f} spacings")
    
    clear_gpu_memory()
    
    return {
        'results': results_sorted,
        'best': results_sorted[0]
    }

# Exécution
comparison_results = compare_all_approaches(gamma)

---

# TEST 5 : Train/Test avec Meilleure Combinaison

In [ ]:
def train_test_validation(gamma: xp.ndarray, lags: List[int], 
                          train_ratio: float = 0.5) -> Dict:
    """
    TEST 5: Validation train/test avec coefficients gelés.
    """
    print("\n" + "="*60)
    print(f"TEST 5 : TRAIN/TEST VALIDATION")
    print(f"Lags: {lags}")
    print("="*60)
    
    N = len(gamma)
    max_lag = max(lags)
    split = int(N * train_ratio)
    
    print(f"\n📊 Split:")
    print(f"   TRAIN: n = {max_lag+1} à {split} ({split - max_lag} points)")
    print(f"   TEST:  n = {split+1} à {N} ({N - split} points)")
    
    # Fit sur TRAIN
    coeffs_train, err_train_pct, err_train_unf = fit_recurrence_gpu(
        gamma, lags, start=max_lag, end=split
    )
    
    print(f"\n🔧 Coefficients (TRAIN):")
    for i, lag in enumerate(lags):
        print(f"   a_{lag} = {float(coeffs_train[i]):.6f}")
    print(f"   c = {float(coeffs_train[-1]):.6f}")
    
    # Test avec coefficients gelés
    indices_test = xp.arange(split, N)
    n_test = len(indices_test)
    
    # Prédictions
    y_pred = xp.zeros(n_test)
    for i, lag in enumerate(lags):
        y_pred += coeffs_train[i] * gamma[indices_test - lag]
    y_pred += coeffs_train[-1]
    
    y_true = gamma[indices_test]
    errors_abs = xp.abs(y_pred - y_true)
    spacings = local_spacing_vec(y_true)
    errors_unf = errors_abs / spacings
    
    err_test_unf = float(xp.mean(errors_unf))
    err_test_pct = float(xp.mean(errors_abs / y_true * 100))
    
    ratio = err_test_unf / err_train_unf
    
    print(f"\n📈 Résultats:")
    print(f"   TRAIN: {err_train_unf:.4f} spacings")
    print(f"   TEST:  {err_test_unf:.4f} spacings")
    print(f"   Ratio: {ratio:.2f}x")
    
    if ratio <= 1.5:
        print(f"\n🎯 VERDICT: ✅ PASS (ratio ≤ 1.5)")
        verdict = "PASS"
    elif ratio <= 2.0:
        print(f"\n🎯 VERDICT: ⚠️ MARGINAL (ratio ≤ 2.0)")
        verdict = "MARGINAL"
    else:
        print(f"\n🎯 VERDICT: ❌ FAIL (ratio > 2.0)")
        verdict = "FAIL"
    
    clear_gpu_memory()
    
    return {
        'verdict': verdict,
        'train_error': err_train_unf,
        'test_error': err_test_unf,
        'ratio': ratio,
        'coefficients': [float(c) for c in coeffs_train]
    }

# Exécution avec la meilleure combinaison trouvée
best_lags = comparison_results['best']['lags']
print(f"\nValidation avec meilleure combinaison: {best_lags}")
traintest_results = train_test_validation(gamma, best_lags)

---

# Synthèse Finale

In [ ]:
def final_synthesis(hybrid_res, log_res, stability_res, comparison_res, traintest_res):
    """
    Synthèse finale de tous les tests.
    """
    print("\n" + "="*70)
    print("SYNTHÈSE PHASE 1 - GPU VALIDATION")
    print("="*70)
    
    print(f"\n📊 RÉSULTATS:")
    print(f"\n   1. HYBRID OPTIMAL: {hybrid_res['best']['lags']}")
    print(f"      Erreur: {hybrid_res['best']['error_unf']:.4f} spacings")
    print(f"      vs baseline [1,2,3,4]: {hybrid_res['baseline']['error_unf']:.4f} spacings")
    
    print(f"\n   2. LOG-CORRECTION: {log_res['verdict']}")
    print(f"      Coeffs avec R² > 0.5: {log_res['high_r2_count']}")
    print(f"      Coeffs proches GIFT: {log_res['close_to_gift_count']}")
    
    print(f"\n   3. COMPARAISON GLOBALE:")
    for i, r in enumerate(comparison_res['results'][:3]):
        print(f"      {i+1}. {r['name']}: {r['error_unf']:.4f} spacings")
    
    print(f"\n   4. TRAIN/TEST: {traintest_res['verdict']}")
    print(f"      Ratio: {traintest_res['ratio']:.2f}x")
    
    # Export JSON
    summary = {
        'n_zeros': int(N_ZEROS),
        'hybrid': {
            'best_lags': hybrid_res['best']['lags'],
            'best_error': hybrid_res['best']['error_unf'],
            'baseline_error': hybrid_res['baseline']['error_unf']
        },
        'log_correction': {
            'verdict': log_res['verdict'],
            'high_r2_count': log_res['high_r2_count'],
            'coefficients': log_res['regression']
        },
        'comparison': {
            'ranking': [{'name': r['name'], 'error': r['error_unf']} 
                       for r in comparison_res['results'][:5]]
        },
        'train_test': {
            'verdict': traintest_res['verdict'],
            'ratio': traintest_res['ratio'],
            'lags': best_lags
        }
    }
    
    print(f"\n" + "="*70)
    print("EXPORT JSON")
    print("="*70)
    print(json.dumps(summary, indent=2, default=float))
    
    return summary

# Synthèse
final_summary = final_synthesis(
    hybrid_results, 
    log_results, 
    stability_results, 
    comparison_results,
    traintest_results
)

In [ ]:
# Sauvegarde des résultats
with open('phase1_gpu_results.json', 'w') as f:
    json.dump(final_summary, f, indent=2, default=float)
print("\n💾 Résultats sauvegardés dans phase1_gpu_results.json")

---

# Visualisations (optionnel)

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Évolution des coefficients
    ax1 = axes[0, 0]
    for r in log_results['regression']:
        n_centers = [w['n_center'] for w in log_results['window_data']]
        ax1.plot(n_centers, r['values'], 'o-', label=r['name'], alpha=0.7)
    ax1.set_xlabel('n (indice du zéro)')
    ax1.set_ylabel('Valeur du coefficient')
    ax1.set_title('Évolution des coefficients par fenêtre')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Coefficient vs 1/log(n)
    ax2 = axes[0, 1]
    for r in log_results['regression']:
        inv_log_n = [w['inv_log_n'] for w in log_results['window_data']]
        ax2.scatter(inv_log_n, r['values'], label=r['name'], alpha=0.7)
        # Ligne de régression
        x_line = np.linspace(min(inv_log_n), max(inv_log_n), 100)
        y_line = r['a_inf'] + r['b'] * x_line
        ax2.plot(x_line, y_line, '--', alpha=0.5)
    ax2.set_xlabel('1/log(n)')
    ax2.set_ylabel('Coefficient')
    ax2.set_title(f'Régression: aᵢ(n) = aᵢ∞ + bᵢ/log(n)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Comparaison des approches
    ax3 = axes[1, 0]
    names = [r['name'] for r in comparison_results['results']]
    errors = [r['error_unf'] for r in comparison_results['results']]
    colors = ['green' if e == min(errors) else 'steelblue' for e in errors]
    ax3.barh(names, errors, color=colors)
    ax3.set_xlabel('Erreur (spacings)')
    ax3.set_title('Comparaison des approches')
    ax3.grid(True, alpha=0.3, axis='x')
    
    # 4. Stabilité par fenêtre
    ax4 = axes[1, 1]
    n_centers = [d['n_center'] for d in stability_results['data']]
    errors_unf = [d['error_unf'] for d in stability_results['data']]
    ax4.plot(n_centers, errors_unf, 'o-', color='purple')
    ax4.set_xlabel('n (indice central)')
    ax4.set_ylabel('Erreur (spacings)')
    ax4.set_title('Erreur par fenêtre')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig('phase1_gpu_visualization.png', dpi=150)
    plt.show()
    print("\n📊 Visualisation sauvegardée dans phase1_gpu_visualization.png")
    
except ImportError:
    print("matplotlib non disponible - visualisation ignorée")